In [ ]:
!pip install llama-index qdrant-client
!pip install pdfminer.six
!pip install llama-index-vector-stores-qdrant
!pip install llama-index-llms-openai llama-index-embeddings-jinaai
!pip install llama-index-llms-huggingface "huggingface_hub[inference]"

  Using cached llama_index_llms_huggingface-0.2.4-py3-none-any.whl (11 kB)


In [ ]:
!pip install --upgrade llama-index

In [ ]:
!pip uninstall llama-index-llms-huggingface -y

Found existing installation: llama-index-llms-huggingface 0.2.4
Uninstalling llama-index-llms-huggingface-0.2.4:
  Successfully uninstalled llama-index-llms-huggingface-0.2.4


In [ ]:
!pip install llama-index-llms-huggingface-api

In [ ]:
hf_inference_api_key = '<HF Key>'

In [ ]:
#!huggingface-cli login

In [ ]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl = (
    "You are a document parser/interpreter for the Human Resources Department.\n"
    "All questions you get come from employees of On-Target Supplies and Logictics"
    "You are given the following context information.\n"
    "---------------------\n"
    "{context_str}\\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query. Please be detailed and complete and provide exact quotes from the documentation from where the information was obtained.\n"
    "If the context information does not contain an answer to the query, "
    "respond with \"No information\".\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt = PromptTemplate(qa_prompt_tmpl)

In [ ]:
jina_emb_api_key = "<Jina Key>"

In [ ]:
import regex as re
from io import BytesIO, StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def document_parser(file_name):
  with open(file_name, 'rb') as pdf_file:
    pdf_data = pdf_file.read()

  text_paras = []
  parser = PDFParser(BytesIO(pdf_data))
  doc = PDFDocument(parser)
  rsrcmgr = PDFResourceManager()
  for page in PDFPage.create_pages(doc):
      output_string = StringIO()
      device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
      interpreter = PDFPageInterpreter(rsrcmgr, device)
      interpreter.process_page(page)
      page_text = output_string.getvalue()
      text_paras.extend(re.split(r'\n\s*\n', page_text))

  return text_paras

In [ ]:
from llama_index.core.readers import StringIterableReader

rag_docs = StringIterableReader().load_data(document_parser("Organizational Guide 01.2019.pdf"))
rag_docs.extend(StringIterableReader().load_data(document_parser("OTSL Travel Policy Effective 2011.pdf")))
rag_docs.extend(StringIterableReader().load_data(document_parser("Performance Mgmt Training Employee View.pdf")))

In [ ]:
for i in range(0,len(rag_docs)-1):
  print(rag_docs[i].text)

2019 
ORGANIZATIONAL 
GUIDE 
As an employee at On-Target Supplies & Logistics, you are an important member of the team. Your employment 
experience will be a positive one as you take the time to become familiar with the Company policies and your 
shared responsibility in the Company’s commitment to excellence. 
This organizational guide does not in any way constitute an agreement between the Company and its employees 
with respect to level of compensation, duration of employment or any other matter. Under no circumstances should 
this guide or the Human Resource Policies and Procedures contained herein be considered a contract of 
employment or a legally binding agreement. 

INTRODUCTION 
This  organizational  guide  has  been  developed  to  provide  employees  of  On-Target 
Supplies & Logistics, Ltd. (“On-Target”), and all of its affiliate companies, with key policies, 
procedures and general practices.  The policies, listed in this guide are not exhaustive, 
but are meant to guide

In [ ]:
qdrant_api_key = "Qdrant Key"
qdrant_server = "Qdrant Server Address"

In [ ]:
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore

client = qdrant_client.QdrantClient(qdrant_server, api_key=qdrant_api_key)
vector_store = QdrantVectorStore(client=client, collection_name="RAG_TRIAL")

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import get_response_synthesizer
from llama_index.core import Settings

from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI


model_trial = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_model  = HuggingFaceInferenceAPI(
    model_name=model_trial,
    token='HF Key'
)

from llama_index.embeddings.jinaai import JinaEmbedding

jina_embedding_model = JinaEmbedding(
    api_key="<Jina Key>",
    model="jina-embeddings-v2-base-en",
)

Settings.llm = llm_model
Settings.embed_model = jina_embedding_model

storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create an index
index = VectorStoreIndex.from_documents(
    rag_docs, storage_context=storage_context)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    text_qa_template=qa_prompt,
    response_mode="compact",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [ ]:
response = query_engine.query("""
how does the company maintain employee records?
""")
print(response.response)


According to the context information, the company maintains employee records in a personnel file that is kept at the corporate office and on the ExponentHR® system. This suggests that the company uses a combination of physical and digital methods to store employee records. However, the specific details of how these records are organized and managed are not provided in the context information.


In [ ]:
response = query_engine.query("""
whats the probationary period for newly hired employees?
""")
print(response.response)


The probationary period for newly hired employees at On-Target Supplies and Logistics is 90 days, as stated in the context information: "NEW HIRE PROBATIONARY PERIOD\n---------------------".


In [ ]:
response = query_engine.query("""
what are the laws that OTSL adheres to with regards to equal employment?
""")
print(response.response)


On-Target Supplies and Logistics adheres to the Rehabilitation Act of 1973 and the Americans with Disabilities Act of 1990 with regards to equal employment opportunity. This information is provided in the context information given.


In [ ]:
response = query_engine.query("""
what are the different steps of OTSL's discipline policy?
""")
print(response.response)


The different steps of On-Target Supplies and Logistics' (OTSL) discipline policy are outlined in the provided context information. They are as follows:

1. Coaching: This is the first step in OTSL's discipline policy. It involves providing feedback and guidance to the employee to help them improve their performance or behavior.

2. Counseling or Training: If the employee's performance or behavior does not improve after coaching, the next step is counseling or training. This step involves providing more structured guidance and support to help the employee correct their behavior or improve their performance.

3. Disciplinary Action: If the employee's performance or behavior still does not improve after counseling or training, the next step is disciplinary action. This step may involve written warnings, suspensions, or other forms of disciplinary action.

4. Termination: If the employee's performance or behavior continues to be unacceptable after disciplinary action, the final step is t

In [ ]:
# @title Default title text
response = query_engine.query("""
describe the workers compensation policy in 5 sentences?
""")
print(response.response)


1. The company's workers' compensation policy generally does not cover injuries sustained while traveling to or from work.
2. It also does not cover injuries that occur during lunch periods.
3. This policy is in line with the general understanding of workers' compensation, which typically applies to injuries that occur during work hours and within the work premises.
4. The policy is designed to protect employees who are injured while performing their job duties.
5. However, it is important to note that the policy has exceptions and specific cases may be evaluated individually.


In [ ]:
response = query_engine.query("""
what are the rules for expenses reimbursements?
""")
print(response.response)


Based on the provided context information, the rules for expense reimbursements at On-Target Supplies and Logistics are as follows:

1. All expenses must be pre-approved by a manager or supervisor before they are incurred.
2. Original receipts are required for all expenses, except for meals and incidental expenses (M&IE) which can be submitted with a legible credit card statement.
3. Expenses must be submitted within 30 days of the expense being incurred.
4. Reimbursement requests will be paid via direct deposit to the employee's bank account.
5. Expenses will be reimbursed at the actual cost incurred, up to the maximum amount allowed by company policy.
6. Alcohol is not a reimbursable expense, except in certain circumstances where it is necessary for business entertainment.
7. Employees are responsible for understanding and complying with all company policies related to expense reimbursements.

Exact quote from the context information:
"All expenses must be pre-approved by a manager 

In [ ]:
response = query_engine.query("""
what exactly is COBRA?
""")
print(response.response)


COBRA stands for Consolidated Omnibus Budget Reconciliation Act. It is a federal law that may allow you to temporarily continue your health coverage after your employment ends, you lose coverage as a dependent of the covered employee, or another qualifying event occurs (as defined by federal law). If you elect COBRA continuation coverage, you will be required to pay the entire premium for coverage, plus a 2% administrative fee. The maximum amount of time you can continue COBRA coverage is 18 or 36 months, depending on the qualifying event. This information is obtained directly from the context information provided, which states: "Given the context information and not prior knowledge, answer the query. Please be detailed and complete and provide exact quotes from the documentation from where the information was obtained. If the context information does not contain an answer to the query, respond with 'No information'." The context information is: "COBRA\n---------------------"


In [ ]:
response = query_engine.query("""
what is the minimum age requirement TO WORK FOR OTSL?
""")
print(response.response)


According to the context information, the minimum age requirement to work for On-Target Supplies and Logistics (OTSL) is 14 years old. However, there are work restrictions for youth under the age of 18 that are governed by Texas and federal child labor laws. Youth under the age of 14 are prohibited to work except under specific circumstances permitted by state and federal law. Youth ages 14 or 15 are restricted to working during the hours, time of day and in the occupations permitted by state and federal law. Youth ages 16 or 17 are restricted from working in occupations that are not permitted by state and federal law. It is recommended to contact Organizational Development for further guidance concerning the employment of youth ages 14-17. (Quote from context information: "Youth under the age of 14 are prohibited to work except under specific circumstances permitted by state and federal law. Youth ages 14 or 15 are restricted to working during the hours, time of day and in the occupa

In [ ]:
response = query_engine.query("""
what's the attendence policy?
""")
print(response.response)


The attendance policy for On-Target Supplies and Logistics is as follows:

"Employees are expected to be on time for all shifts. If an employee is unable to report to work, they must notify their supervisor as soon as possible. Failure to do so will result in disciplinary action.

Employees are allowed three (3) occurrences of unexcused absences or tardiness in a rolling twelve (12) month period. After the third occurrence, the employee will receive a written warning. Further occurrences will result in additional disciplinary action, up to and including termination."

This information can be found in the "Policy" section of the context information provided.


In [ ]:
response = query_engine.query("""
how is the pay schedule for employees at OTSL?
""")
print(response.response)


The pay schedule for employees at On-Target Supplies and Logistics (OTSL) is either weekly, semi-monthly, or monthly. Weekly payments are made on Friday, while semi-monthly payments are made on the 1st and 15th of each month. Monthly payments are also made on the 1st of each month. This information is provided in the context information given: "Employees are paid weekly, on Friday; semi-monthly on the 1st and 15th; or monthly on the 1st."


In [ ]:
response = query_engine.query("""
how much will the OTSL reimburse for stays, meals and travel if I need to travel overnight for business reasons?
""")
print(response.response)


The context information does not contain an answer to this query. There is no mention of reimbursement for stays, meals, or travel in the provided document. It only mentions that OTSL will reimburse employees for mileage directly related to business use at a rate of $0.50 per mile, as well as certain out-of-pocket expenses including tolls and parking.


In [ ]:
response = query_engine.query("""
what does on-target suggest for cost effective travel practices?
""")
print(response.response)


According to the context information, On-Target Supplies and Logistics suggests cost effective travel practices for its employees to manage business travel. This is stated in the purpose of the travel policy which is to "set guidelines for employees to manage business travel in the most cost-effective and efficient manner that will maximize productivity during business travel." However, the context information does not provide specific cost effective travel practices that the employees should follow.


In [ ]:
response = query_engine.query("""
where can i find my performance reviews?
""")
print(response.response)


According to the context information, you can find your performance reviews by accessing the OTSL Performance Management Process. The document does not provide specific details on where the performance reviews are located within the process, but it does mention that the process includes a section on "Accessing Your Review". Therefore, you may be able to find your performance reviews within this section.

Quote:
"OTSL Performance Management Process: Accessing Your Review" (title of the document)
"Accessing Your Review" (section heading within the document)
